# Conexão com o BigQuery

- Objetivo: conectar ao bigquery, testar os comando de criação de tabela e overwrite e lapidar para a pipeline

In [3]:
# 1. Importa o SDK e configura o projeto (opcional)
from google.cloud import bigquery
import os

In [4]:
# Definindo o projeto
os.environ['GOOGLE_CLOUD_PROJECT'] = 'rj-smas-dev'

In [5]:
# 2. Cria o cliente
client = bigquery.Client()

In [6]:
# 3. Lista os datasets no projeto
datasets = list(client.list_datasets())
if datasets:
    print("✔️ Datasets encontrados no projeto:")
    for ds in datasets:
        print(f"  • {ds.dataset_id}")
else:
    print("⚠️ Nenhum dataset encontrado. Verifique PROJECT_ID e credenciais.")

Forbidden: 403 GET https://bigquery.googleapis.com/bigquery/v2/projects/rj-smas-dev/datasets?prettyPrint=false: Caller does not have required permission to use project rj-smas-dev. Grant the caller the roles/serviceusage.serviceUsageConsumer role, or a custom role with the serviceusage.services.use permission, by visiting https://console.developers.google.com/iam-admin/iam/project?project=rj-smas-dev and then retry. Propagation of the new permission may take a few minutes. [{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'USER_PROJECT_DENIED', 'domain': 'googleapis.com', 'metadata': {'consumer': 'projects/rj-smas-dev', 'containerInfo': 'rj-smas-dev', 'service': 'bigquery.googleapis.com'}}, {'@type': 'type.googleapis.com/google.rpc.LocalizedMessage', 'locale': 'en-US', 'message': 'Caller does not have required permission to use project rj-smas-dev. Grant the caller the roles/serviceusage.serviceUsageConsumer role, or a custom role with the serviceusage.services.use permi

# Conexão com o Arcgis

- Objetivo: conectar a feature, analisar os dados e lapidar para a pipeline

In [9]:
from arcgis.gis import GIS
from arcgis.features import FeatureLayer
import pandas as pd

***Conexão com o SIURB***

In [8]:
gis_siurb = GIS("https://siurb.rio/portal", "SMAS_ed01", "@smas#25")
item_siurb = gis.content.get("6832ff4ca54c4608b169682ae3a5b088")

NameError: name 'gis' is not defined

***Conexão com o AGOL***

In [10]:
gis_agol = GIS("https://www.arcgis.com/index.html", "smds.adm", "#smds.adm@25")
item_agol = gis_agol.content.get("1ef5fb0ea56c42849d338bb30d796b0f")

***Teste da conexão***

In [12]:
# Configurar entre siurb e agol 
item = item_agol

print("Título:", item.title)
print("Layers :", [lyr.properties.name   for lyr in item.layers])
print("Tables :", [tbl.properties.name   for tbl in item.tables])

Título: ABORDAGEM SOCIAL - CPSR (2023.2)
Layers : ['Ficha de Abordagem Social - SMAS', 'repeat_abordagem']
Tables : []


# 2.1 Ficha de Abordagem Social – SMAS

- Layer index 0: `"Ficha de Abordagem Social - SMAS"`

***Conectando para pegar os dados no arcgis***

In [16]:
# já temos `item = gis.content.get(...)`
layer_smas = item.layers[0]  
print("URL da layer:", layer_smas.url)

# consulta sem geometria, pegando só as colunas
fl = layer_smas.query(
    where="1=1",
    out_fields="*",
    return_geometry=False,
    max_records=5
)

# converte para pandas
df_smas = fl.sdf  
print("Linhas × Colunas:", df_smas.shape)
display(df_smas.head())

URL da layer: https://pgeo3.rio.rj.gov.br/arcgis/rest/services/Hosted/service_38e6b1b9fd64490da151470cb0739fe9/FeatureServer/0


KeyboardInterrupt: 

In [ ]:
# Lista todos os campos definidos no serviço
fields = [fld["name"] for fld in layer_smas.properties.fields]
print("Total de campos no serviço:", len(fields))
print(fields)


Total de campos no serviço: 42
['objectid', 'globalid', 'uniquerowid', 'unidade_calculo', 'unidade_bairro', 'unidade_cas', 'filtro_primeira_letra_equip', 'filtro_cinco_letra_equip', 'nome_usuario', 'nome_social', 'data_nascimento', 'data_nascimento_iso', 'idade', 'faixa_etaria', 'cpf', 'calc_valido', 'motivo_cpf', 'estado_nascimento', 'migrante_sim_nao', 'nome_mae', 'nome_pai', 'grupo_familiar', 'raca_cor_etnia', 'sexo', 'filtro_primeira_letra', 'filtro_data_abordagem', 'filtro_mes_ultima_abord', 'filtro_ano_ultima_abordagem', 'flag_painel', 'exclusao_unidade_calculo', 'exclusao_unidade_bairro', 'exclusao_unidade_cas', 'nome_usuario_ver', 'excluir_ficha', 'nome_tecnico_preenc_form', 'observacoes_edicao', 'data_preenc_form', 'data_exclusao', 'created_user', 'created_date', 'last_edited_user', 'last_edited_date']


**Pontos de inspeção**  
- Número de colunas (`df_smas.shape[1]`)  
- Tipos de cada coluna (`df_smas.dtypes`)  
- Valores nulos (`df_smas.isna().sum()`)


In [ ]:
# detalhes rápidos
print(df_smas.dtypes)
print("\nValores ausentes por coluna:")
print(df_smas.isna().sum())


objectid                                Int64
globalid                       string[python]
uniquerowid                    string[python]
unidade_calculo                string[python]
unidade_bairro                 string[python]
unidade_cas                    string[python]
filtro_primeira_letra_equip    string[python]
filtro_cinco_letra_equip       string[python]
nome_usuario                   string[python]
nome_social                    string[python]
data_nascimento                string[python]
data_nascimento_iso            string[python]
idade                                   Int32
faixa_etaria                   string[python]
cpf                            string[python]
calc_valido                    string[python]
motivo_cpf                     string[python]
estado_nascimento              string[python]
migrante_sim_nao               string[python]
nome_mae                       string[python]
nome_pai                       string[python]
grupo_familiar                 str

## Processo de ELT

**Movimento dos dados para o Bigquery**

- Objetivo: Levar os dados para o Bigquery para serem tratados por lá

## Processo de ETL

**Tratamento dos dados coletados**

- Objetivo: Tratar os dados que vieram do arcgis, preparar para a subido pro Bigquery

In [32]:
# Seleciona e trata as colunas que vão para o BigQuery
import pandas as pd
import re
from datetime import date

COLS_KEEP = [
    "globalid",
    "unidade_calculo",
    "nome_usuario",
    "data_nascimento",
    "cpf",
    "nome_mae",
    "filtro_ano_ultima_abordagem",
    "excluir_ficha",
    "created_user",
]

# 1️⃣ Copia apenas as colunas relevantes
df_out = df_smas[COLS_KEEP].copy()

# 2️⃣ Função auxiliar → date | None
def to_date_obj(col):
    dt = pd.to_datetime(col, errors="coerce", dayfirst=True)
    # .dt.date devolve objeto python date; NaT → NaN, então trocamos p/ None
    return dt.dt.date.where(~dt.isna(), None)

# 3️⃣ Limpezas e normalizações
df_out["data_nascimento"] = to_date_obj(df_out["data_nascimento"])

df_out["cpf"] = (
    df_out["cpf"]
      .astype("string")
      .str.replace(r"\D", "", regex=True)          # só dígitos
      .replace("", pd.NA)                          # string vazia → NA
)

df_out["excluir_ficha"] = (
    df_out["excluir_ficha"]
      .str.strip()
      .str.lower()
      .map({"sim": "Sim", "não": "Não", "nao": "Não"})
      .fillna("Não")                                # default
      .astype("string")                            
)

# 4️⃣ Garante StringDtype nas demais colunas
for col in [
    "globalid",
    "unidade_calculo",
    "nome_usuario",
    "nome_mae",
    "filtro_ano_ultima_abordagem",
    "created_user",
]:
    df_out[col] = df_out[col].astype("string")

# 5️⃣ Inspeção rápida
display(df_out.head())
print(df_out.dtypes)
print("Linhas:", len(df_out))

,globalid,unidade_calculo,nome_usuario,data_nascimento,cpf,nome_mae,filtro_ano_ultima_abordagem,excluir_ficha,created_user
0,{E1D6732E-87FF-4A13-A43A-3EAA66ACB6FB},EQUIPE 24H (ESPECIALIZADA),WESLEYDA SILVA RODRIGUES,1995-08-02,<NA>,OTELINA DA SILVA RODRIGUES,<NA>,Não,datarioadmin
1,{AE1FBD81-15EF-4D87-B0A1-6E9F594A3889},CREAS PROFESSORA ALDAIZA SPOSATI,GILMAR DA SILVA,1986-01-20,<NA>,MARIA DE FATIA ELIAS DA CRUZ,<NA>,Não,datarioadmin
2,{1D7D5549-18BF-4637-8C0F-8D9D78C8E328},EQUIPE 24H (ESPECIALIZADA),DIEGO DE SOUZA MOREIRA,1992-12-08,<NA>,ROSELI DE SOUZA MOREIA,<NA>,Não,datarioadmin
3,{D6A02F5F-0556-4A1F-9829-A791D8C7181D},EQUIPE 24H (ESPECIALIZADA),VICENTE DE PAIVA,1957-09-06,<NA>,JANDIRA HORTENCIA,<NA>,Não,datarioadmin
4,{D2B0BCA0-7376-429A-8750-4EB51C645E6B},EQUIPE 24H (ESPECIALIZADA),RODRIGO MARTINS DE MAGALHAES,1997-10-19,<NA>,MARTA MARTINS,<NA>,Não,datarioadmin


globalid                       string[python]
unidade_calculo                string[python]
nome_usuario                   string[python]
data_nascimento                        object
cpf                            string[python]
nome_mae                       string[python]
filtro_ano_ultima_abordagem    string[python]
excluir_ficha                  string[python]
created_user                   string[python]
dtype: object
Linhas: 169419


**Subida dos dados para o BigQuery**
- Objetivo: esquematizar as tabelas do bq e subir os dados tratados

In [37]:
# Sobe df_out (preparado no BLOCO 1) para o BigQuery
import subprocess
from google.oauth2.credentials import Credentials
from google.cloud import bigquery

# 1️⃣ Pega um access-token da conta que já fez `gcloud auth login`
access_token = subprocess.check_output(
    ["gcloud", "auth", "print-access-token"],
    text=True,
).strip()

# 2️⃣ Instancia credenciais *sem* refresh nem quota-project
creds = Credentials(token=access_token)

# 3️⃣ Cria o cliente BigQuery usando essas credenciais
PROJECT_ID  = "rj-smas-dev"          # defina explicitamente
DATASET_ID  = "teste_abordagem"
TABLE_ID    = "abordagem_sintetica"
TABLE_REF   = f"{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}"

client = bigquery.Client(project=PROJECT_ID, credentials=creds)

# 4️⃣ (opcional) cria dataset via API; se falhar, crie no CLI: `bq mk`
try:
    client.create_dataset(DATASET_ID, exists_ok=True)
except Exception as e:
    print("⚠️  Não consegui criar o dataset via API – "
          "crie no CLI se precisar:", e)

# 5️⃣ esquema da tabela
schema = [
    bigquery.SchemaField("globalid", "STRING"),
    bigquery.SchemaField("unidade_calculo", "STRING"),
    bigquery.SchemaField("nome_usuario", "STRING"),
    bigquery.SchemaField("data_nascimento", "DATE"),
    bigquery.SchemaField("cpf", "STRING"),
    bigquery.SchemaField("nome_mae", "STRING"),
    bigquery.SchemaField("filtro_ano_ultima_abordagem", "STRING"),
    bigquery.SchemaField("excluir_ficha", "STRING"),
    bigquery.SchemaField("created_user", "STRING"),
]

job_config = bigquery.LoadJobConfig(
    schema=schema,
    write_disposition="WRITE_TRUNCATE",
)

# 6️⃣ carrega o df_out (gerado no BLOCO 1)
load_job = client.load_table_from_dataframe(df_out, TABLE_REF, job_config=job_config)
load_job.result()          # espera terminar

table = client.get_table(TABLE_REF)
print(f"✔️ {table.full_table_id} com {table.num_rows} linhas.")

/root/venv-abordagem/lib/python3.11/site-packages/google/cloud/bigquery/_pandas_helpers.py:489: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


✔️ rj-smas-dev:teste_abordagem.abordagem_sintetica com 169419 linhas.


# 2.2 repeat_abordagem

- Layer index 1: `"repeat_abordagem"`

In [13]:
# já temos `item_x = gis.content.get(...)`
layer_smas = item.layers[1]  
print("URL da layer:", layer_smas.url)

# consulta sem geometria, pegando só as colunas
fl = layer_smas.query(
    where="1=1",
    out_fields="*",
    return_geometry=False,
)

# converte para pandas
df_smas = fl.sdf  
print("Linhas × Colunas:", df_smas.shape)
display(df_smas.head())

URL da layer: https://services1.arcgis.com/OlP4dGNtIcnD3RYf/arcgis/rest/services/service_ab1e5fb472de491ca105077fe17b72ea/FeatureServer/1
Linhas × Colunas: (228712, 266)


,objectid,globalid,repeat_unidade_calculo,repeat_unidade_bairro,repeat_unidade_cas,repeat_nome_usuario,repeat_nome_social,repeat_data_nascimento,repeat_data_nascimento_iso,repeat_idade,...,observacao,parentglobalid,CreationDate,Creator,EditDate,Editor,flag_exc_repeat,note_creas,repeat_cpf,repeat_flag_painel
0,730,aba4bfab-8851-433e-80c8-f44b70d67de8,CENTRO POP JOSÉ SARAMAGO,Bonsucesso,4,EDMILSON DE LIMA,<NA>,12/09/1983,1983-09-12,39,...,<NA>,20ae622e-4af8-4417-96bb-6854e424a9dd,2023-07-05 15:19:32.519999,subpse.siurb.territorio,2023-09-01 19:17:00.960999,smds.adm,<NA>,CREAS Nelson Carneiro,<NA>,<NA>
1,733,edb5a6c2-737c-4789-ab8e-dd6ece1ed0b1,CENTRO POP JOSÉ SARAMAGO,Bonsucesso,4,FABIO RAMOS DA SILVA,<NA>,14/07/1981,1981-07-14,41,...,<NA>,cefaad8f-7094-4f41-b0a3-40826fd76835,2023-07-05 15:37:20.377000,subpse.siurb.territorio,2023-09-01 19:17:01.762000,smds.adm,<NA>,CREAS Nelson Carneiro,<NA>,<NA>
2,734,aa649e86-4996-4d0a-bf3b-76f7ab9282b0,CENTRO POP JOSÉ SARAMAGO,Bonsucesso,4,JORGE FRANCISCO DOS SANTOS,<NA>,13/01/1964,1964-01-13,60,...,<NA>,b2c6ae6a-f7c1-406c-9d3f-316f37527e2b,2023-07-05 15:46:13.423000,subpse.siurb.territorio,2024-12-27 14:52:34.986000,smds.adm,<NA>,CREAS Nelson Carneiro,<NA>,<NA>
3,735,46df03af-d8cf-45a4-9a01-bdd3e2e9d6d4,CENTRO POP JOSÉ SARAMAGO,Bonsucesso,4,NAULDOBERTO CIRINO,<NA>,28/05/1974,1974-05-28,49,...,<NA>,8009fb97-0fc4-46ec-a934-2267ea5fff54,2023-07-05 15:52:25.684000,subpse.siurb.territorio,2023-09-01 19:17:03.829999,smds.adm,<NA>,CREAS Nelson Carneiro,<NA>,<NA>
4,736,52db383b-14cf-4f0a-b0e6-4cfc2e2bec6d,CENTRO POP JOSÉ SARAMAGO,Bonsucesso,4,LUIZ FERNANDO DE JESUS,<NA>,01/05/1993,1993-05-01,30,...,<NA>,6f94111c-7ef7-4e99-8198-ca520fde37e0,2023-07-05 16:05:15.941999,subpse.siurb.territorio,2023-09-01 19:17:04.812000,smds.adm,<NA>,CREAS Nelson Carneiro,<NA>,<NA>


## Processo de ELT

**Movimento dos dados para o Bigquery**

- Objetivo: Levar os dados para o Bigquery para serem tratados por lá

## Processo de ETL

**Tratamento dos dados coletados**

- Objetivo: Tratar os dados que vieram do arcgis, preparar para a subido pro Bigquery

In [41]:
# Seleciona e trata as colunas que vão para o BigQuery
import pandas as pd
import re
from datetime import date

COLS_KEEP = [
    "objectid",
    "globalid",
    "repeat_unidade_calculo",
    "turno_abordagem",
    "data_abordagem",
    "dia_num_data_abordagem",
    "mes_abrev_data_abordagem",
    "ano_num_data_abordagem",
    "bairro_abord",
    "x",
    "y",
    "resp_abordagem",
    "resp_abordagem1",
    "aceita_acolhimento",
    "parentrowid",
    "created_user",
]

# 1️⃣ Copia apenas as colunas relevantes
df_out = df_smas[COLS_KEEP].copy()

# 2️⃣ Função auxiliar → date | None
def to_date_obj(col):
    dt = pd.to_datetime(col, errors="coerce", dayfirst=True)
    # .dt.date devolve objeto python date; NaT → NaN, então trocamos p/ None
    return dt.dt.date.where(~dt.isna(), None)

# 3️⃣ Limpezas e normalizações
df_out["data_abordagem"] = to_date_obj(df_out["data_abordagem"])

df_out["coordenadas"] = df_out["y"].astype(str) + ", " + df_out["x"].astype(str)

# 4️⃣ Garante StringDtype nas demais colunas
for col in [
    "objectid",
    "globalid",
    "repeat_unidade_calculo",
    "turno_abordagem",
    "dia_num_data_abordagem",
    "mes_abrev_data_abordagem",
    "ano_num_data_abordagem",
    "bairro_abord",
    "x",
    "y",
    "resp_abordagem",
    "resp_abordagem1",
    "aceita_acolhimento",
    "parentrowid",
    "created_user",
    "coordenadas",
]:
    df_out[col] = df_out[col].astype("string")

# 5️⃣ Inspeção rápida
display(df_out.head())
print(df_out.dtypes)
print("Linhas:", len(df_out))

,objectid,globalid,repeat_unidade_calculo,turno_abordagem,data_abordagem,dia_num_data_abordagem,mes_abrev_data_abordagem,ano_num_data_abordagem,bairro_abord,x,y,resp_abordagem,resp_abordagem1,aceita_acolhimento,parentrowid,created_user,coordenadas
0,217487,{748D7B39-61B5-41F0-9A17-CFD9B5F4A1EA},CREAS PROFESSORA ALDAIZA SPOSATI,noite,2025-03-28,28,mar,2025,Bangu,-43.46272009,-22.87452501,creas,itinerante,nao,{F0321C2D-5314-4E2A-BD32-81A75EC77A6C},SMAS.subpse.territorio_ed01,"-22.87452501, -43.46272009"
1,217488,{0DAF5C7A-A96D-49E3-AEA0-6C967D8D0976},EQUIPE 24H (ESPECIALIZADA),tarde,2025-04-04,04,abr,2025,Cidade Nova,-43.20369104,-22.91012883,cgppsr,abord_itinerante,sim,{AA31417F-4A51-400E-BF87-705B651BEEEB},SMAS.subpse.territorio_ed01,"-22.91012883, -43.20369104"
2,217489,{2307E9FE-D996-48E9-8DFB-BB02D4832FDB},EQUIPE 24H (ESPECIALIZADA),manha,2025-04-04,04,abr,2025,Leme,-43.16505788,-22.96156698,cgppsr,abord_itinerante,nao,{53A7EF74-5176-4391-934C-D6312299CC67},SMAS.subpse.territorio_ed01,"-22.96156698, -43.16505788"
3,217490,{C5DBCA8A-AEB4-4660-A37B-4F957BED6013},EQUIPE 24H (ESPECIALIZADA),noite,2025-04-04,04,abr,2025,Tijuca,-43.2329918,-22.92533655,cgppsr,abord_itinerante,nao,{26843F25-DBB9-4A81-9064-BF646960E383},SMAS.subpse.territorio_ed01,"-22.92533655, -43.2329918"
4,217491,{0B44666C-B799-4828-86C7-1E74713B7680},EQUIPE 24H (ESPECIALIZADA),noite,2025-04-04,04,abr.,2025,Tijuca,681189.80409656,7463664.90550056,cgppsr,abord_itinerante,nao,{29A0CBFE-5BBF-4198-85D4-C1CFA89C7ECD},SMAS.subpse.territorio_ed01,"7463664.90550056, 681189.80409656"


objectid                    string[python]
globalid                    string[python]
repeat_unidade_calculo      string[python]
turno_abordagem             string[python]
data_abordagem                      object
dia_num_data_abordagem      string[python]
mes_abrev_data_abordagem    string[python]
ano_num_data_abordagem      string[python]
bairro_abord                string[python]
x                           string[python]
y                           string[python]
resp_abordagem              string[python]
resp_abordagem1             string[python]
aceita_acolhimento          string[python]
parentrowid                 string[python]
created_user                string[python]
coordenadas                 string[python]
dtype: object
Linhas: 275728


**Subida dos dados para o BigQuery**
- Objetivo: esquematizar as tabelas do bq e subir os dados tratados

In [43]:
# Sobe df_out (preparado no BLOCO 1) para o BigQuery
import subprocess
from google.oauth2.credentials import Credentials
from google.cloud import bigquery

# 1️⃣ Pega um access-token da conta que já fez `gcloud auth login`
access_token = subprocess.check_output(
    ["gcloud", "auth", "application-default", "print-access-token"],
    text=True,
).strip()

# 2️⃣ Instancia credenciais *sem* refresh nem quota-project
creds = Credentials(token=access_token)

# 3️⃣ Cria o cliente BigQuery usando essas credenciais
PROJECT_ID  = "rj-smas-dev"          # defina explicitamente
DATASET_ID  = "teste_abordagem"
TABLE_ID    = "repeat_siurb"
TABLE_REF   = f"{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}"

client = bigquery.Client(project=PROJECT_ID, credentials=creds)

# 4️⃣ (opcional) cria dataset via API; se falhar, crie no CLI: `bq mk`
try:
    client.create_dataset(DATASET_ID, exists_ok=True)
except Exception as e:
    print("⚠️  Não consegui criar o dataset via API – "
          "crie no CLI se precisar:", e)

# 5️⃣ esquema da tabela
schema = [
    bigquery.SchemaField("objectid", "STRING"),
    bigquery.SchemaField("globalid", "STRING"),
    bigquery.SchemaField("repeat_unidade_calculo", "STRING"),
    bigquery.SchemaField("turno_abordagem", "STRING"),
    bigquery.SchemaField("data_abordagem", "DATE"),  # objeto datetime convertido no carregamento
    bigquery.SchemaField("dia_num_data_abordagem", "STRING"),
    bigquery.SchemaField("mes_abrev_data_abordagem", "STRING"),
    bigquery.SchemaField("ano_num_data_abordagem", "STRING"),
    bigquery.SchemaField("bairro_abord", "STRING"),
    bigquery.SchemaField("x", "STRING"),
    bigquery.SchemaField("y", "STRING"),
    bigquery.SchemaField("resp_abordagem", "STRING"),
    bigquery.SchemaField("resp_abordagem1", "STRING"),
    bigquery.SchemaField("aceita_acolhimento", "STRING"),
    bigquery.SchemaField("parentrowid", "STRING"),
    bigquery.SchemaField("created_user", "STRING"),
    bigquery.SchemaField("coordenadas", "STRING"),
]

job_config = bigquery.LoadJobConfig(
    schema=schema,
    write_disposition="WRITE_TRUNCATE",
)

# 6️⃣ carrega o df_out (gerado no BLOCO 1)
load_job = client.load_table_from_dataframe(df_out, TABLE_REF, job_config=job_config)
load_job.result()          # espera terminar

table = client.get_table(TABLE_REF)
print(f"✔️ {table.full_table_id} com {table.num_rows} linhas.")

/root/venv-abordagem/lib/python3.11/site-packages/google/cloud/bigquery/_pandas_helpers.py:489: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


✔️ rj-smas-dev:teste_abordagem.repeat_siurb com 275728 linhas.
